# Crawl

This notebook contains an implementation of Crawl, an experimental graph-based method for building recommender systems.

## What does this algorithm do?

The algorithm takes a set of items as input and outputs an extension of this set that contains additional items which are most likely to belong to it, based on previous transaction data.

## Sampling the dataset

In this demonstration, the transaction data is playlists belonging to the [Spotify Million Playlist Dataset](https://www.aicrowd.com/challenges/spotify-million-playlist-dataset-challenge), and the items are artists.

Since the original dataset is 1 million playlists long, we will select a random sample of 100 thousand playlists for running our experiment.

In [1]:
import json

In [2]:
data_file = open("extended-data/filtered-data.json")
data = json.load(data_file)
info_file = open("extended-data/artist-info.json")
info = json.load(info_file)

In [3]:
TOTAL_PLAYLISTS = len(data)
NUM_ARTISTS = len(info)

In [4]:
NUM_PLAYLISTS = 100000

In [5]:
import random

In [6]:
random.seed(30)

In [7]:
select = []
for i in range(TOTAL_PLAYLISTS):
    select.append(i)
select = random.sample(select, NUM_PLAYLISTS)
sample = []
for i in select:
    sample.append(data[i])

## Constructing the similarity graph

The next step is to construct the similarity graph.  
The vertices of this graph represent artists and the weights of each edge denotes the similarity of the two artists, which is denoted by a measure we have devised called modified-lift.

$$modified-lift(A, B) = \frac{support(\{A, B\})}{\sqrt{support(\{A\}) * support(\{B\})}}$$

Edges are only constructed between the most similar artists - the limit is defined by the `WEIGHT_THRESHOLD` hyperparameter.  
Since the entire complete graph cannot be constructed at once and pruned, it is constructed batch wise.  
The `BATCH_SIZE` hyperparameter is used to control this batch size.  
In order to prevent cases os isolated vertices and vertices with too many edges, the `MIN_DEGREE` and `MAX_DEGREE` hyperparameters have been introduced.

In [8]:
BATCH_SIZE = 1000
WEIGHT_THRESHOLD = 0.2
MIN_DEGREE = 5
MAX_DEGREE = 50

In [9]:
count = []
for i in range(NUM_ARTISTS):
    count.append(0)
for playlist in sample:
    for item in playlist:
        count[item] += 1

In [10]:
def init_graph(graph):
    for _ in range(NUM_ARTISTS):
        graph.append([])

In [11]:
def build_graph(graph, start, end):
    sub = []
    for i in range(start, end):
        sub.append([])
        for j in range(NUM_ARTISTS):
            sub[i - start].append([0, j])
    for playlist in sample:
        for item in playlist:
            if (item >= start) and (item < end):
                for comp in playlist:
                    sub[item - start][comp][0] += 1
    for i in range(start, end):
        for j in range(NUM_ARTISTS):
            sub[i - start][j][0] /= (count[i] * count[j]) ** 0.5
        sub[i - start] = sorted(sub[i - start], reverse=True)
        for j in range(len(sub[i - start])):
            if ((sub[i - start][j][0] >= WEIGHT_THRESHOLD) or (j < MIN_DEGREE)) and (j < MAX_DEGREE):
                graph[i].append(sub[i - start][j])
            else:
                break

In [12]:
graph = []
init_graph(graph)
for start in range(0, NUM_ARTISTS, BATCH_SIZE):
    build_graph(graph, start, min(start + BATCH_SIZE, NUM_ARTISTS))

## Function for printing playlists

In [13]:
def print_playlist(playlist):
    for idx, item in enumerate(playlist):
        print(f"{idx + 1}:\t{info[item]['name']}")

## The algorithm

The algorithm works by running a modified version of the Dijkstra's algorithm for finding shortest paths from the vertices representing artists that are present in the original playlist.  
The similarity of a path is considered to be the product of weights of its constituent edges.  
This is analagous to how the distance of a path in traditional applications of shortest-path algorithms is considered to be the sum of the weights of its edges.  
The algorithm also takes an optional argument called `weights`, which denotes the relative importance of each artist in the sample playlist.

In [14]:
import heapdict

In [15]:
def extend(playlist, size, weights=[]):
    playlist = playlist.copy()
    if weights == []:
        for _ in range(len(playlist)):
            weights.append(1)
    flag = []
    for _ in range(NUM_ARTISTS):
        flag.append(False)
    for item in playlist:
        flag[item] = True
    pq = heapdict.heapdict()
    for idx, item in enumerate(playlist):
        for pair in graph[item]:
            if not flag[pair[1]]:
                if pair[1] in pq:
                    if -pair[0] < pq[pair[1]]:
                        pq[pair[1]] = -(pair[0] * weights[idx])
                else:
                    pq[pair[1]] = -(pair[0] * weights[idx])
    while (len(playlist) < size):
        if len(pq) == 0:
            break
        next = list(pq.popitem())
        next[1] = (-next[1]) ** 0.5
        playlist.append(next[0])
        flag[next[0]] = True
        for pair in graph[next[0]]:
            if not flag[pair[1]]:
                if pair[1] in pq:
                    if -(pair[0] * next[1]) < pq[pair[1]]:
                        pq[pair[1]] = -(pair[0] * next[1])
                else:
                    pq[pair[1]] = -(pair[0] * next[1])
    return playlist

## Testing with EDM artists

We now test the performance of the algorithm by providing it with a playlist of some artists belong to the genre of EDM (Electronic Dance Music).  
Ideally, the algorithm must extend the playlist in such a way that other EDM artists are present at the top.

In [16]:
edm_playlist = [66, 90, 432, 466, 614]
print_playlist(edm_playlist)

1:	Avicii
2:	Martin Garrix
3:	Alan Walker
4:	Swedish House Mafia
5:	Armin van Buuren


In [17]:
extended_edm_playlist = extend(edm_playlist, 20)
print_playlist(extended_edm_playlist)

1:	Avicii
2:	Martin Garrix
3:	Alan Walker
4:	Swedish House Mafia
5:	Armin van Buuren
6:	The Chainsmokers
7:	Zedd
8:	Calvin Harris
9:	Major Lazer
10:	David Guetta
11:	DJ Snake
12:	Hardwell
13:	Tiësto
14:	Galantis
15:	Kygo
16:	Steve Aoki
17:	Kaskade
18:	Sebastian Ingrosso
19:	Jonas Blue
20:	Afrojack


As you can see from the results, the algorithm does indeed add EDM artists.  
It was able to correctly identify the nature of the sample playlist without being explicitly told about it, and by merely looking at transaction data.

## Testing with Country artists

In [18]:
country_playlist = [110, 167, 171, 177, 821]
print_playlist(country_playlist)

1:	Florida Georgia Line
2:	Blake Shelton
3:	Tim McGraw
4:	Keith Urban
5:	Alabama


In [19]:
extended_country_playlist = extend(country_playlist, 20)
print_playlist(extended_country_playlist)

1:	Florida Georgia Line
2:	Blake Shelton
3:	Tim McGraw
4:	Keith Urban
5:	Alabama
6:	Luke Bryan
7:	Dierks Bentley
8:	Thomas Rhett
9:	Jason Aldean
10:	Kenny Chesney
11:	Jake Owen
12:	Eric Church
13:	Chris Young
14:	Lee Brice
15:	Brad Paisley
16:	Cole Swindell
17:	Billy Currington
18:	Sam Hunt
19:	Brett Eldredge
20:	Zac Brown Band


## Testing with Hip-Hop and Latin artists

In [20]:
hiphop_latin_playlist = [2, 27, 112, 189, 281]
print_playlist(hiphop_latin_playlist)

1:	Kanye West
2:	Eminem
3:	Shakira
4:	Enrique Iglesias
5:	J Balvin


In [21]:
extended_hiphop_latin_playlist = extend(hiphop_latin_playlist, 20)
print_playlist(extended_hiphop_latin_playlist)

1:	Kanye West
2:	Eminem
3:	Shakira
4:	Enrique Iglesias
5:	J Balvin
6:	Drake
7:	Maluma
8:	Nicky Jam
9:	JAY Z
10:	Kendrick Lamar
11:	Big Sean
12:	Wisin
13:	Daddy Yankee
14:	Lil Wayne
15:	Farruko
16:	J. Cole
17:	Zion & Lennox
18:	Yandel
19:	Future
20:	Rae Sremmurd


## Testing with Hip-Hop and Latin artists but biased towards Hip-Hop

In [22]:
biased_hiphop_latin_playlist = extend(hiphop_latin_playlist, 20, [0.4, 0.3, 0.1, 0.1, 0.1])
print_playlist(biased_hiphop_latin_playlist)

1:	Kanye West
2:	Eminem
3:	Shakira
4:	Enrique Iglesias
5:	J Balvin
6:	Future
7:	Migos
8:	Lil Uzi Vert
9:	21 Savage
10:	Kodak Black
11:	A Boogie Wit da Hoodie
12:	Travis Scott
13:	Gucci Mane
14:	Post Malone
15:	Playboi Carti
16:	Young Thug
17:	Rae Sremmurd
18:	Big Sean
19:	Drake
20:	Kendrick Lamar
